<a href="https://colab.research.google.com/github/poenzel/sofina/blob/test/Untitled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extraction de datasets (csv) à partir de pdf
Ce notebook emploie des méthodes de détection de tables afin d'extraire les informations. Passer par des modèles de ML entrainés est dû au grand nombre de formats différents employant des cellules mergées, ainsi que la présence de watermark dans ces pdfs qui recouvrent les tables.

## Ordre des opérations :    
### 1. Charger le(s) pdf(s)
### 2. Transformer chaque page du(des) pdf(s) en image
### 3. Détecter sur l'image les tables
### 4. Identifier les cellules ainsi que leur contenu
### 5. S'assurer que les cellules soient alignés afin d'avoir un tableau cohérent
### 6. Merge le tout

## Après l'extraction :
Une fois les données à disposition, des étapes de cleaning supplémentaires seront encore nécessaires. Cependant, le contenu des cellules sera correct. Il faudra juste éliminer les termes parasites restants (par exemple, '.com' qui apparait à cause de la watermark) et adapter le format si nécessaire.

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
!pip install pdf2image
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [49]:
from pdf2image import convert_from_path

In [50]:
images = convert_from_path('/content/drive/MyDrive/Sofina/Dataset1/ANDREESSEN & HOROWITZ/AH VI-Q L.P. - 2019 Q4 Audited Financials.pdf')

In [51]:
!mkdir -p 'drive/MyDrive/Sofina/Dataset1/ANDREESSEN & HOROWITZ/pages'

In [52]:
 for i in range(len(images)):
  images[i].save('drive/MyDrive/Sofina/Dataset1/ANDREESSEN & HOROWITZ/pages/page'+str(i)+'.jpg','JPEG')

In [53]:
!wget https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl
!pip install -U layoutparser-0.0.0-py3-none-any.whl

--2024-01-25 12:54:45--  https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.46.61, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19145360 (18M) [application/octet-stream]
Saving to: ‘layoutparser-0.0.0-py3-none-any.whl.1’

layoutparser-0.0.0- 100%[===================>]  18.26M  4.87MB/s    in 18s     

2024-01-25 12:55:05 (1.03 MB/s) - ‘layoutparser-0.0.0-py3-none-any.whl.1’ saved [19145360/19145360]

Processing ./layoutparser-0.0.0-py3-none-any.whl
layoutparser is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [54]:
!python3 -m pip install paddlepaddle-gpu
!pip install "paddleocr>=2.0.1"
!pip install protobuf==3.20.0
!git clone https://github.com/PaddlePaddle/PaddleOCR.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.0
    Uninstalling protobuf-3.20.0:
      Successfully uninstalled protobuf-3.20.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.15.1 requires protobuf<4.24,>=3.19.6, but you have protobuf 4.25.2 which is incompatible.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.25.2 which is incompatible.
  Using cached protobuf-3.20.0-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.2
    Uninstalling protobuf-4.25.2:
      Successfully uninstalled protobuf-4.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are instal

fatal: destination path 'PaddleOCR' already exists and is not an empty directory.


In [160]:
import tensorflow as tf
import numpy as np
import cv2
import layoutparser as lp
import pandas as pd

In [56]:
 import cv2
 import layoutparser as lp
 image = cv2.imread("drive/MyDrive/Sofina/Dataset1/ANDREESSEN & HOROWITZ/pages/page5.jpg")
 image = image[..., ::-1]

 model = lp.PaddleDetectionLayoutModel(config_path= "lp://PubLayNet/ppyolov2_r50vd_dcn_365e_publaynet/config",
                                       threshold = 0.5,
                                       label_map= {0: "Text", 1: "Title", 2: "List", 3: "Table", 4:"Figure"},
                                       enforce_cpu = True,
                                       enable_mkldnn= True)
 layout = model.detect(image)

In [57]:
layout

Layout(_blocks=[TextBlock(block=Rectangle(x_1=208.52716064453125, y_1=284.2144775390625, x_2=1995.473876953125, y_2=1469.7366943359375), text=None, id=None, type=Figure, parent=None, next=None, score=0.9376512765884399), TextBlock(block=Rectangle(x_1=639.96875, y_1=1500.1741943359375, x_2=1556.133056640625, y_2=1534.4515380859375), text=None, id=None, type=Text, parent=None, next=None, score=0.8913436532020569)], page_data={})

In [58]:
x_1 = 208.52716064453125
y_1=284.2144775390625
x_2=1995.473876953125
y_2=1469.7366943359375

In [59]:
for l in layout :
    x_1 = int(l.block.x_1)
    y_1 = int(l.block.y_1)
    x_2 = int(l.block.x_2)
    y_2 = int(l.block.y_2)
    break

In [60]:
print(x_1,y_1,x_2,y_2)

208 284 1995 1469


In [61]:
im2 = cv2.imread('drive/MyDrive/Sofina/Dataset1/ANDREESSEN & HOROWITZ/pages/page5.jpg')

In [62]:
cv2.imwrite('drive/MyDrive/Sofina/Dataset1/ANDREESSEN & HOROWITZ/pages/ext_im.jpg', im2[y_1:y_2,x_1:x_2])

True

In [63]:
from paddleocr import PaddleOCR, draw_ocr

In [64]:
ocr = PaddleOCR(lang='en', use_gpu = False)
image_path = 'drive/MyDrive/Sofina/Dataset1/ANDREESSEN & HOROWITZ/pages/ext_im.jpg'
image_cv = cv2.imread(image_path)
image_height = image_cv.shape[0]
image_width = image_cv.shape[1]
output = ocr.ocr(image_path)

[2024/01/25 12:56:03] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='

In [ ]:
boxes = [line[0] for line in output[0]]
texts = [line[1][0] for line in output[0]]
probabilities = [line[1][1] for line in output[0]]
# for text in texts :
  # print(text)

In [66]:
image_boxes = image_cv.copy()

In [67]:
for box,text in zip(boxes,texts):
  cv2.rectangle(image_boxes, (int(box[0][0]),int(box[0][1])), (int(box[2][0]),int(box[2][1])),(0,0,255),1)
  cv2.putText(image_boxes, text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,1,(222,0,0),1)

In [68]:
cv2.imwrite('drive/MyDrive/Sofina/Dataset1/ANDREESSEN & HOROWITZ/pages/detections.jpg', image_boxes)

True

### Reconstruction

In [69]:
im = image_cv.copy()

In [70]:
horiz_boxes = []
vert_boxes = []

for box in boxes:
  x_h, x_v = 0,int(box[0][0])
  y_h, y_v = int(box[0][1]),0

  # Différence entre l'abscisse min et max de la boite
  width_h,width_v = image_width, int(box[2][0]-box[0][0])
  # Pareil mais entre les ordonnées
  height_h,height_v = int(box[2][1]-box[0][1]),image_height

  horiz_boxes.append([x_h,y_h,x_h+width_h,y_h+height_h])
  vert_boxes.append([x_v,y_v,x_v+width_v,y_v+height_v])

  cv2.rectangle(im,(x_h,y_h), (x_h+width_h,y_h+height_h),(0,0,255),1)
  cv2.rectangle(im,(x_v,y_v), (x_v+width_v,y_v+height_v),(0,255,0),1)


In [71]:
cv2.imwrite('drive/MyDrive/Sofina/Dataset1/ANDREESSEN & HOROWITZ/pages/corrected_boxes.jpg',im)

True

In [72]:
# On garde que les boxes avec le meilleur score
horiz_out = tf.image.non_max_suppression(
    horiz_boxes,
    probabilities,
    max_output_size = 1000,
    iou_threshold=0.1,
    score_threshold=float('-inf'),
    name=None
)

In [73]:
horiz_lines = np.sort(np.array(horiz_out))
print(horiz_lines)

[  0   4   8  12  19  20  21  22  28  33  38  39  54  58  59  61  66  71
  72  74  80  85  91  97  99 108 117 128 130]


In [104]:
im_nms = image_cv.copy()

In [105]:
for val in horiz_lines:
  cv2.rectangle(im_nms, (int(horiz_boxes[val][0]),int(horiz_boxes[val][1])), (int(horiz_boxes[val][2]),int(horiz_boxes[val][3])),(0,0,255),1)


In [106]:
cv2.imwrite('drive/MyDrive/Sofina/Dataset1/ANDREESSEN & HOROWITZ/pages/im_nms.jpg',im_nms)

True

In [120]:
# Pareil pour les boxes verticales
vert_out = tf.image.non_max_suppression(
    vert_boxes,
    probabilities,
    max_output_size = 11, # On pourrait aller chercher le max table size
    iou_threshold=0.1,
    score_threshold=float('-inf'),
    name=None
)


In [121]:
vert_lines = np.sort(np.array(vert_out))
print(vert_lines)

[  0   7   8   9  10  12  13  14  16  21  26  28  29  48 126 128]


In [122]:
for val in vert_lines:
  cv2.rectangle(im_nms, (int(vert_boxes[val][0]),int(vert_boxes[val][1])), (int(vert_boxes[val][2]),int(vert_boxes[val][3])),(255,0,0),1)


In [123]:
cv2.imwrite('drive/MyDrive/Sofina/Dataset1/ANDREESSEN & HOROWITZ/pages/im_nms.jpg',im_nms)

True

### Transforme en table

In [156]:
out_array = [["" for i in range(len(vert_lines))] for j in range(len(horiz_lines))]
print(np.array(out_array).shape)
print(out_array)

(29, 16)
[['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], 

In [124]:
unordered_boxes = []

for i in vert_lines:
  print(vert_boxes[i])
  unordered_boxes.append(vert_boxes[i][0])

[1534, 0, 1586, 1185]
[1160, 0, 1256, 1185]
[1647, 0, 1755, 1185]
[383, 0, 443, 1185]
[529, 0, 579, 1185]
[821, 0, 866, 1185]
[938, 0, 1026, 1185]
[1117, 0, 1162, 1185]
[1402, 0, 1446, 1185]
[13, 0, 99, 1185]
[916, 0, 933, 1185]
[1074, 0, 1089, 1185]
[1359, 0, 1376, 1185]
[1753, 0, 1768, 1185]
[700, 0, 756, 1185]
[1292, 0, 1340, 1185]


In [125]:
ordered_boxes = np.argsort(unordered_boxes)
print(ordered_boxes)

[ 9  3  4 14  5 10  6 11  7  1 15 12  8  0  2 13]


In [126]:
def intersection(box_1, box_2):
  return [box_2[0], box_1[1],box_2[2], box_1[3]]

In [127]:
def iou(box_1, box_2):

  x_1 = max(box_1[0], box_2[0])
  y_1 = max(box_1[1], box_2[1])
  x_2 = min(box_1[2], box_2[2])
  y_2 = min(box_1[3], box_2[3])

  inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
  if inter == 0:
      return 0

  box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
  box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))

  return inter / float(box_1_area + box_2_area - inter)

In [155]:
len(vert_lines)

16

In [157]:
for i in range(len(horiz_lines)):
  for j in range(len(vert_lines)):
    resultant = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]] )

    for b in range(len(boxes)):
      the_box = [boxes[b][0][0],boxes[b][0][1],boxes[b][2][0],boxes[b][2][1]]
      if(iou(resultant,the_box)>0.1):
        out_array[i][j] = texts[b]

0 13
1 2
1 5
1 6
1 8
1 9
1 14
2 2
2 4
2 6
2 9
2 14
3 1
3 2
3 3
3 4
3 6
3 8
3 9
3 10
3 12
3 13
3 14
4 0
5 0
6 0
7 0
8 0
8 1
8 4
8 5
8 6
8 7
8 11
8 12
8 13
9 0
9 1
9 4
9 6
9 12
9 13
10 0
11 0
11 2
11 3
11 4
11 6
11 8
11 9
11 10
11 12
11 13
11 15
12 0
12 2
12 3
12 4
12 6
12 8
12 9
12 10
12 12
12 13
13 0
14 0
15 0
15 1
15 4
15 6
15 12
15 13
16 0
16 4
16 6
16 12
16 13
17 0
18 0
19 0
19 1
19 4
19 6
19 12
19 13
20 0
20 1
20 4
20 6
20 12
20 13
21 0
21 1
21 4
21 6
21 12
21 13
22 0
23 0
23 2
23 3
23 4
23 6
23 12
23 13
24 0
24 2
24 3
24 4
24 6
24 12
24 13
25 0
25 2
25 3
25 4
25 6
25 8
25 9
25 10
25 12
25 13
26 0
26 2
26 3
26 4
26 6
26 7
26 8
26 9
26 10
26 12
26 13
27 0
27 3
27 6
27 10
27 13
28 10
28 11


In [158]:
out_array=np.array(out_array)

In [161]:
pd.DataFrame(out_array).to_csv('drive/MyDrive/Sofina/Dataset1/ANDREESSEN & HOROWITZ/pages/sample.csv')

## En dessous :
Tentatives d'exploiter les coordonnées obtenues. Un des problèmes vient de la watermark...

In [81]:
!pip install tabula-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 3.0 MB/s eta 0:00:00


In [82]:
!pip install img2table

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 3.9 MB/s eta 0:00:00


In [83]:
# import tabula
from img2table.document import Image

path = '/content/drive/MyDrive/Sofina/Dataset1/ANDREESSEN & HOROWITZ/pages/ext_im.jpg'

img = Image(src=path)

# Table identification
img_tables = img.extract_tables()

# Result of table identification
img_tables
# custom_area = (208.52716064453125-104, 284.2144775390625-142, 1995.473876953125+104, 1469.7366943359375+142)

#TODO : Split la merged cell pcq même avec une custom area ça passe pas

# dfs = tabula.read_pdf(path, stream = True,
#                       # area= custom_area,
#                       pages= '6')

# dfs[0].head(20)

[]